In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/My Drive/Colab Notebooks

/content/drive/My Drive/Colab Notebooks


In [3]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler

The general parameters for our task are the future horizon of our prediction and the hyperparameter for the network

In [4]:
prediction_days = 30
nof_units = 4

We will encapsulate our model creation step in a function. It will accept a single parameter, units, which is the dimension of the innter cells in LSTM

In [5]:
def create_model(nunits):

    # Initialising the RNN
    regressor = Sequential()

    # Adding the input layer and the LSTM layer
    regressor.add(LSTM(units = nunits, activation = 'sigmoid', input_shape = (None, 1)))

    # Adding the output layer
    regressor.add(Dense(units = 1))

    # Compiling the RNN
    regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')
    
    return regressor

In [7]:
# load the data and encode the date
df = pd.read_csv("bitstampUSD_1-min_data_2012-01-01_to_2021-03-31.csv")
df['date'] = pd.to_datetime(df['Timestamp'],unit='s').dt.date
group = df.groupby('date')
Real_Price = group['Weighted_Price'].mean()
# We will predict the average daily price that is why we need to group the date

In [8]:
# The next step will be splitting the data into training and testing dataset
df_train= Real_Price[:len(Real_Price)-prediction_days] # This will start the dataset at 0 until len(Real_price)-prediction_days
df_test= Real_Price[len(Real_Price)-prediction_days:] # This will start the dataset at  len(Real_price)-prediction_days until the end

In [9]:
# Perform preprocessing
training_set = df_train.values
training_set = np.reshape(training_set, (len(training_set), 1))

sc = MinMaxScaler()
training_set = sc.fit_transform(training_set)
X_train = training_set[0:len(training_set)-1]  # training set
y_train = training_set[1:len(training_set)] # testing dataset
X_train = np.reshape(X_train, (len(X_train), 1, 1))

In [10]:
# Fit the model

# Initialising the RNN
regressor = create_model(nunits = 4)
# Fitting the RNN to the Training set
regressor.fit(X_train, y_train, batch_size = 5, epochs = 100) # We will run for a 100 times

Epoch 1/100
669/669 [==============================] - 2s 2ms/step - loss: 0.0108
Epoch 2/100
669/669 [==============================] - 1s 2ms/step - loss: 0.0069
Epoch 3/100
669/669 [==============================] - 1s 2ms/step - loss: 0.0042
Epoch 4/100
669/669 [==============================] - 1s 2ms/step - loss: 0.0021
Epoch 5/100
669/669 [==============================] - 1s 2ms/step - loss: 8.2680e-04
Epoch 6/100
669/669 [==============================] - 1s 2ms/step - loss: 2.4661e-04
Epoch 7/100
669/669 [==============================] - 1s 2ms/step - loss: 1.0196e-04
Epoch 8/100
669/669 [==============================] - 1s 2ms/step - loss: 8.0572e-05
Epoch 9/100
669/669 [==============================] - 1s 2ms/step - loss: 7.5712e-05
Epoch 10/100
669/669 [==============================] - 2s 3ms/step - loss: 7.2738e-05
Epoch 11/100
669/669 [==============================] - 1s 2ms/step - loss: 6.6466e-05
Epoch 12/100
669/669 [==============================] - 1s 2ms/step 

As can be seen from the above result, once the number of epochs are increased, the loss is decreased. That means the model is learning good with high accuracy.